In [61]:
import os
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages TargetHolding:pyspark-cassandra:0.3.5,org.apache.spark:spark-streaming-kafka_2.11:1.6.1 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-csv_2.11:1.4.0,org.apache.spark:spark-streaming-kafka_2.11:1.6.1,com.datastax.spark:spark-cassandra-connector_2.11:1.6.0-M2 pyspark-shell'
import time

In [12]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
sc.stop()
conf = SparkConf() \
    .setAppName("Streaming test") \
    .setMaster("local[2]") \
    .set("spark.cassandra.connection.host", "127.0.0.1")
sc = SparkContext(conf=conf) 


In [13]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
#import pyspark_cassandra

In [76]:

#sc.stop()
sc = SparkContext("local[2]", "KafkaReceiveStream") 
ssc = StreamingContext(sc, 5)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Streaming test, master=local[2]) created by __init__ at <ipython-input-38-a603e2b12da6>:5 

In [ ]:
kvs = KafkaUtils.createStream(ssc, "127.0.0.1:2181", "spark-streaming-consumer", {'test': 1})
lines = kvs.map(lambda x: x[1])
content = lines.map(lambda line: line.split(',')).repartition(20).count()
content.pprint()
ssc.start()

In [ ]:
ssc.stop()

In [77]:
rdd = sc.cassandraTable("ksfraud", "rawtrx22")

AttributeError: 'SparkContext' object has no attribute 'cassandraTable'

In [14]:
sqlContext=SQLContext(sc)

In [87]:


dd=sqlContext.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(table="rawdata", keyspace="tx")\
    .load()

In [88]:
dd.count()

100000

In [8]:
dd.registerTempTable("txData");

In [79]:
from pyspark.sql.types import *
fields = [StructField("card_pan_id", StringType(), False),\
          StructField("term_miduid", StringType(), False),\
          StructField("term_mid", StringType(), False),\
          StructField("term_uid", StringType(), False),\
          StructField("term_mcc", StringType(), False),\
          StructField("term_country", StringType(), False),\
          StructField("tx_amount", StringType(), False),\
          StructField("tx_datetime", StringType(), False),\
          StructField("tx_accepted", StringType(), False),\
          StructField("tx_ecom", StringType(), False),\
          StructField("tx_3d_secure", StringType(), False),\
          StructField("tx_emv", StringType(), False),\
          StructField("age", StringType(), False),\
          StructField("language", StringType(), False),\
          StructField("gender", StringType(), False),\
          StructField("broker", StringType(), False),\
          StructField("card_brand", StringType(), False),\
          StructField("tx_fraud", StringType(), False),\
        ]
schema = StructType(fields)

In [80]:
csvfile = sqlContext.read.format('com.databricks.spark.csv',).schema(schema).load("20160322stream_subset100.csv")
csvfile.registerTempTable("txData");
csvfile.printSchema()

root
 |-- card_pan_id: string (nullable = false)
 |-- term_miduid: string (nullable = false)
 |-- term_mid: string (nullable = false)
 |-- term_uid: string (nullable = false)
 |-- term_mcc: string (nullable = false)
 |-- term_country: string (nullable = false)
 |-- tx_amount: string (nullable = false)
 |-- tx_datetime: string (nullable = false)
 |-- tx_accepted: string (nullable = false)
 |-- tx_ecom: string (nullable = false)
 |-- tx_3d_secure: string (nullable = false)
 |-- tx_emv: string (nullable = false)
 |-- age: string (nullable = false)
 |-- language: string (nullable = false)
 |-- gender: string (nullable = false)
 |-- broker: string (nullable = false)
 |-- card_brand: string (nullable = false)
 |-- tx_fraud: string (nullable = false)



In [132]:
start = time.time()
df=sqlContext.sql("select * from txData limit 100000")
df=df.repartition(10)
df.count()
df.cache()
end = time.time()
print(end - start)


87.0941560268


In [131]:
start = time.time()
df.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('overwrite')\
    .options(table="rawdata", keyspace="tx")\
    .save()
end = time.time()
print(end - start)


10.4327499866
